In [ ]:
import pandas as pd
pd.options.display.max_rows = None
import numpy as np
from modules.functions import create_home_and_away_simple_dataframe,plot_game_trends
from modules.functions import get_agg_boxscore,get_game_timestamp_half, add_pie_to_dataframe,get_schedule
from IPython.display import clear_output
import time

In [ ]:
get_schedule("Purdue")

In [ ]:
game_id = 401589296

n_tries = 0

try:
    test_df
except NameError as e:
    test_df = pd.DataFrame()

last_reported_time = None

while n_tries < 10:

    
    timestamp,half = get_game_timestamp_half(game_id)

    if timestamp != last_reported_time:
        n_tries = 0
        clear_output()
        last_reported_time = timestamp
        temp_df = get_agg_boxscore(game_id = game_id,disp = True)
        temp_df = temp_df.transpose().rename_axis("Team").reset_index()
        temp_df['Half'] = half
        temp_df['Timestamp'] = timestamp
        test_df = pd.concat([test_df,temp_df])
        create_home_and_away_simple_dataframe(game_id,disp = True)
        time.sleep(np.random.randint(12,17))
    else:
        time.sleep(25)
        n_tries +=1

In [ ]:
plot_game_trends(test_df,half = half,color1='black',color2='green')

In [ ]:
df1,df2 = create_home_and_away_simple_dataframe(game_id)
display(add_pie_to_dataframe(df1))
display(add_pie_to_dataframe(df2))